In [1]:
import sys
import os
sys.path.append("../src")
import llm_utils
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

classes = ["War/Terror", "Conspiracy Theory", "Education", "Election Campaign", "Environment", 
              "Government/Public", "Health", "Immigration/Integration", 
              "Justice/Crime", "Labor/Employment", 
              "Macroeconomics/Economic Regulation", "Media/Journalism", "Religion", "Science/Technology"]

oa_without_context_elaboration_first_v04_df = pd.read_csv("../data/openassistant_llama_30b_4bit/generic_prompt_without_context_elaboration_first_v04/generic_test_0.csv")
extraction_function = llm_utils.get_extraction_function("extract_using_class_token", 1)
oa_without_context_elaboration_first_v04_predictions_per_class, confusion_matrices, classification_reports = llm_utils.calculate_binary_metrics(oa_without_context_elaboration_first_v02_df, classes, extraction_function)
oa_without_context_elaboration_first_v04 = {"confusion_matrices": confusion_matrices, "classification_reports": classification_reports}

TypeError: 'NoneType' object is not callable

In [2]:
import sys
import os
sys.path.append("../src")
import llm_utils
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import ast
import numpy as np


def classification_reports_to_df(classification_reports, binary):

    if binary:
        # Your code for creating the DataFrame and adding the results
        df = pd.DataFrame(columns=['label', 'f1_score_macro', #'precision_macro', 'recall_macro', 'support_macro',
                                        'f1_score_class_0','support_class_0',
                                        'f1_score_class_1', 'support_class_1'])

        for label, cr in classification_reports.items():
            try: 
                df = df.append({
                    'label': label,
                    'f1_score_macro': cr['macro avg']['f1-score'],
                    #'precision_macro': cr['macro avg']['precision'],
                    #'recall_macro': cr['macro avg']['recall'],
                    #'support_macro': cr['macro avg']['support'],
                    'f1_score_class_0': cr['0']['f1-score'],
                    #'precision_class_0': cr['0']['precision'],
                    #'recall_class_0': cr['0']['recall'],
                    'support_class_0': cr['0']['support'],
                    'f1_score_class_1': cr['1']['f1-score'],
                    #'precision_class_1': cr['1']['precision'],
                    #'recall_class_1': cr['1']['recall'],
                    'support_class_1': cr['1']['support']
                }, ignore_index=True)
            except Exception as e:
                print(f"Error for {label}: {e}")
                df = df.append({
                    'label': label,
                    'f1_score_macro': None,
                    #'precision_macro': None,
                    #'recall_macro': None,
                    #'support_macro': None,
                    'f1_score_class_0': None,
                    #'precision_class_0': None,
                    #'recall_class_0': None,
                    'support_class_0': None,
                    'f1_score_class_1': None,
                    #'precision_class_1': None,
                    #'recall_class_1': None,
                    'support_class_1': None
                }, ignore_index=True)
                continue

        # Display the results
        return df
    
    return None

def extract_multilabel_list(classification_str, classes):
    if classification_str is np.nan:
        return ["Others"]
    classification_str = classification_str.replace("[", "").replace("]", "").replace("'", "")
    classification_str = classification_str.split(", ")
    for class_ in classification_str:
        if class_ not in " ".join(classes):
            #print(class_)
            classification_str.remove(class_)
    if classification_str == []:
        classification_str = ["Others"]
    return list(set(classification_str))

def extract_multilabel_list_only_first_class(classification_str, classes):
    if classification_str is np.nan:
        return ["Others"]
    classification_str = classification_str.replace("[", "").replace("]", "").replace("'", "")
    classification_str = classification_str.split(", ")
    for class_ in classification_str:
        if class_ not in " ".join(classes):
            #print(class_)
            classification_str.remove(class_)
    if classification_str == []:
        classification_str = ["Others"]
    classification_str = [classification_str[0]]
    return list(set(classification_str))

def to_dataframe(average_reports, classes):
    data = {
        "precision": [],
        "recall": [],
        "f1-score": [],
        "support": []
    }
    index = []

    for i, average_report in enumerate(average_reports):
        for class_name, metrics in average_report.items():
            if class_name in {'micro avg', 'macro avg', 'weighted avg', 'accuracy'}:
                continue
            index.append(classes[i])
            data["precision"].append(metrics["0"]["precision"])
            data["recall"].append(metrics["0"]["recall"])
            data["f1-score"].append(metrics["0"]["f1-score"])
            data["support"].append(metrics["0"]["support"])

    return pd.DataFrame(data, index=index)

classes = ["War/Terror", "Conspiracy Theory", "Education", "Election Campaign", "Environment", 
              "Government/Public", "Health", "Immigration/Integration", 
              "Justice/Crime", "Labor/Employment", 
              "Macroeconomics/Economic Regulation", "Media/Journalism", "Religion", "Science/Technology", "Others"]
    
#df1 = pd.read_csv("../data/vicuna_4bit/lora/multilabel_with_rules/valid_generic_test_0.csv")
df2 = pd.read_csv("../data/vicuna_4bit/lora/multilabel_without_context_v01/test_generic_test_0.csv")
vicuna_lora_multilabel_without_context_v01_df = df2#pd.concat([df1, df2])
# Cleaning and extracting multilabel classes
vicuna_lora_multilabel_without_context_v01_df['annotations'] = vicuna_lora_multilabel_without_context_v01_df['annotations'].apply(lambda x: extract_multilabel_list(x, classes))
vicuna_lora_multilabel_without_context_v01_df['testmultilabel_without_context_v01'] = vicuna_lora_multilabel_without_context_v01_df['testmultilabel_without_context_v01'].apply(lambda x: extract_multilabel_list(x, classes))

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report

# Transform the labels into binary format
mlb = MultiLabelBinarizer(classes=classes)

# Prepare ground truth and predictions
y_true = mlb.fit_transform(vicuna_lora_multilabel_without_context_v01_df['annotations'])
y_pred = mlb.transform(vicuna_lora_multilabel_without_context_v01_df['testmultilabel_without_context_v01'])

#import pandas as pd
from sklearn.metrics import classification_report

# Generate the classification report
report = classification_report(y_true, y_pred, output_dict=True, target_names=classes)

# Convert the report to a pandas DataFrame
df_report = pd.DataFrame(report).transpose()

print(df_report)

                                    precision    recall  f1-score  support
War/Terror                           0.500990  0.992157  0.665789    255.0
Conspiracy Theory                    0.368421  0.777778  0.500000     45.0
Education                            0.322581  0.769231  0.454545     13.0
Election Campaign                    0.766667  0.696970  0.730159     33.0
Environment                          0.458333  0.785714  0.578947     14.0
Government/Public                    0.572082  0.859107  0.686813    291.0
Health                               0.345455  0.826087  0.487179     46.0
Immigration/Integration              0.315217  0.805556  0.453125     36.0
Justice/Crime                        0.413681  0.927007  0.572072    137.0
Labor/Employment                     0.243590  0.678571  0.358491     28.0
Macroeconomics/Economic Regulation   0.212996  0.951613  0.348083     62.0
Media/Journalism                     0.102564  0.916667  0.184486     48.0
Religion                 

/media/bruno/0d2f61d2-2b9c-4043-9a46-8e4dfe74fc95/bruno/Documents/ip6-twitter-disinformation/.conda/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:895: UserWarning: unknown class(es) ['China', 'Civilian Casualties', 'Diplomacy', 'Entertainment', 'Event', 'Expression', 'Festival', 'Foreign Relations', 'Generosity', 'Genocide', 'Israel/Palestine', 'Judaism', 'Kurdish Forces', 'Military', 'Miscellaneous', 'Music', 'Nightlife', 'Optical Disk', 'Pandemics', 'Party', 'Party and Celebration', 'Partying', 'Performance', 'Political Platform', 'Politics', 'President', 'Productivity', 'Propaganda', 'Referendum...', 'Syria', 'Television', 'U.S.', 'US Domestic Policy', 'US International Relations', 'USA', 'United States', 'Voting', 'Wages/Labors', 'War', 'Welfare/Development', 'Welfare/Social Macroregulation', 'Wildlife', 'World Order', 'Youth'] will be ignored
  warnings.warn(


In [3]:
df2 = pd.read_csv("../data/vicuna_4bit/lora/multilabel_with_rules_v02/test_generic_test_0.csv")
vicuna_lora_multilabel_without_context_v01_df = df2#pd.concat([df1, df2])
# Cleaning and extracting multilabel classes
vicuna_lora_multilabel_without_context_v01_df['annotations'] = vicuna_lora_multilabel_without_context_v01_df['annotations'].apply(lambda x: extract_multilabel_list(x, classes))
vicuna_lora_multilabel_without_context_v01_df['testmultilabel_with_rules_v02'] = vicuna_lora_multilabel_without_context_v01_df['testmultilabel_with_rules_v02'].apply(lambda x: extract_multilabel_list(x, classes))

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report

# Transform the labels into binary format
mlb = MultiLabelBinarizer(classes=classes)

# Prepare ground truth and predictions
y_true = mlb.fit_transform(vicuna_lora_multilabel_without_context_v01_df['annotations'])
y_pred = mlb.transform(vicuna_lora_multilabel_without_context_v01_df['testmultilabel_with_rules_v02'])

#import pandas as pd
from sklearn.metrics import classification_report

# Generate the classification report
report = classification_report(y_true, y_pred, output_dict=True, target_names=classes)

# Convert the report to a pandas DataFrame
df_report = pd.DataFrame(report).transpose()

print(df_report)

                                    precision    recall  f1-score  support
War/Terror                           0.386018  0.996078  0.556407    255.0
Conspiracy Theory                    0.340206  0.733333  0.464789     45.0
Education                            0.191489  0.692308  0.300000     13.0
Election Campaign                    0.628571  0.666667  0.647059     33.0
Environment                          0.190476  0.857143  0.311688     14.0
Government/Public                    0.547521  0.910653  0.683871    291.0
Health                               0.271605  0.956522  0.423077     46.0
Immigration/Integration              0.222973  0.916667  0.358696     36.0
Justice/Crime                        0.354223  0.948905  0.515873    137.0
Labor/Employment                     0.132075  0.750000  0.224599     28.0
Macroeconomics/Economic Regulation   0.180380  0.919355  0.301587     62.0
Media/Journalism                     0.095918  0.979167  0.174721     48.0
Religion                 

/media/bruno/0d2f61d2-2b9c-4043-9a46-8e4dfe74fc95/bruno/Documents/ip6-twitter-disinformation/.conda/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:895: UserWarning: unknown class(es) ['', '#Collision kills taxi driver near Petroport on Queen Nandi Dr', '#NoCandidates = ‘ Government/Public ', '#Olympics', '#TeamUWI #TrendingNow', '@USER', 'Accident', 'Analysis', 'Architecture', 'Asia', 'Aviation', 'Behna', 'Brexit', 'Civilian', 'Collaboration', 'Conspiracy', 'Cuba', 'Death', 'Defeating_ISIS', 'Energy', 'Fake News', 'Geopolitics', 'Govern', 'Government/', 'Haftar', 'Happy birthday!', 'International Relations', 'Jabhat_al_Nusra', 'Justice', 'Justice/Cr', 'Labor/Emp', 'Leader', 'Mac', 'Macroeconom', 'Macroeconomics', 'Macroeconomics/', 'Mammals', 'Media_coverage_of_war', 'Military', 'Military_group', 'Miscellaneous', 'Murder', 'Murder City', 'No referral', 'None', 'None of the above', 'None of these topics seem to be clearly defined as election campaigns or political subjects

In [6]:
df2 = pd.read_csv("../data/vicuna_4bit/lora/multilabel_no_context_v01_256_rank/test_generic_test_0.csv")
vicuna_lora_multilabel_without_context_v01_df = df2#pd.concat([df1, df2])
# Cleaning and extracting multilabel classes
vicuna_lora_multilabel_without_context_v01_df['annotations'] = vicuna_lora_multilabel_without_context_v01_df['annotations'].apply(lambda x: extract_multilabel_list(x, classes))
vicuna_lora_multilabel_without_context_v01_df['testmultilabel_no_context_v01_256_rank'] = vicuna_lora_multilabel_without_context_v01_df['testmultilabel_no_context_v01_256_rank'].apply(lambda x: extract_multilabel_list(x, classes))

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report

# Transform the labels into binary format
mlb = MultiLabelBinarizer(classes=classes)

# Prepare ground truth and predictions
y_true = mlb.fit_transform(vicuna_lora_multilabel_without_context_v01_df['annotations'])
y_pred = mlb.transform(vicuna_lora_multilabel_without_context_v01_df['testmultilabel_no_context_v01_256_rank'])

#import pandas as pd
from sklearn.metrics import classification_report

# Generate the classification report
report = classification_report(y_true, y_pred, output_dict=True, target_names=classes)

# Convert the report to a pandas DataFrame
df_report = pd.DataFrame(report).transpose()

print(df_report)

                                    precision    recall  f1-score  support
War/Terror                           0.492188  0.988235  0.657106    255.0
Conspiracy Theory                    0.447368  0.755556  0.561983     45.0
Education                            0.257143  0.692308  0.375000     13.0
Election Campaign                    0.750000  0.818182  0.782609     33.0
Environment                          0.450000  0.642857  0.529412     14.0
Government/Public                    0.603491  0.831615  0.699422    291.0
Health                               0.368932  0.826087  0.510067     46.0
Immigration/Integration              0.370370  0.833333  0.512821     36.0
Justice/Crime                        0.447183  0.927007  0.603325    137.0
Labor/Employment                     0.250000  0.678571  0.365385     28.0
Macroeconomics/Economic Regulation   0.241379  0.903226  0.380952     62.0
Media/Journalism                     0.128743  0.895833  0.225131     48.0
Religion                 

/media/bruno/0d2f61d2-2b9c-4043-9a46-8e4dfe74fc95/bruno/Documents/ip6-twitter-disinformation/.conda/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:895: UserWarning: unknown class(es) ['', 'Agriculture', 'Belgium', 'Charity', 'China', 'Climate Change', 'Collaboration', 'Corporations', 'Culture', 'Dance', 'Death', 'Debate', 'Dream', 'European Union', 'Execution', 'Fire', 'Global', 'Globalization', 'Heavy Machinery', 'Imagination', 'Irony', 'Journalism', 'Kurdish Forces', 'Latin America', 'Left-wing Politics', 'Like', 'Middle East', 'Mindset', 'Minorities', 'Music', 'Musical Genius', 'Nationality', 'Nature', 'Network', 'Other', 'Pakistan', 'Performance', 'Person', 'Political Science', 'Politician', 'Politics', 'Positive', 'Presidential Election', 'Presidential Power', 'Radio Show', 'Real estate', 'Rebellions', 'Recommendation', 'Regulation', 'Relationship', 'Relationships', 'Review', 'Russia', 'Science', 'Secretary of State', 'Security', 'Self-Improvement', 'Sexuality', 'Sle

In [13]:
df2 = pd.read_csv("../data/vicuna_4bit/lora/multilabel_with_rules_v02_256_rank/test_generic_test_0.csv")
vicuna_lora_multilabel_without_context_v01_df = df2#pd.concat([df1, df2])
# Cleaning and extracting multilabel classes
vicuna_lora_multilabel_without_context_v01_df['annotations'] = vicuna_lora_multilabel_without_context_v01_df['annotations'].apply(lambda x: extract_multilabel_list(x, classes))
vicuna_lora_multilabel_without_context_v01_df['testmultilabel_with_rules_v02_256_rank'] = vicuna_lora_multilabel_without_context_v01_df['testmultilabel_with_rules_v02_256_rank'].apply(lambda x: extract_multilabel_list(x, classes))

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report

# Transform the labels into binary format
mlb = MultiLabelBinarizer(classes=classes)

# Prepare ground truth and predictions
y_true = mlb.fit_transform(vicuna_lora_multilabel_without_context_v01_df['annotations'])
y_pred = mlb.transform(vicuna_lora_multilabel_without_context_v01_df['testmultilabel_with_rules_v02_256_rank'])

#import pandas as pd
from sklearn.metrics import classification_report

# Generate the classification report
report = classification_report(y_true, y_pred, output_dict=True, target_names=classes)

# Convert the report to a pandas DataFrame
df_report = pd.DataFrame(report).transpose()

print(df_report)

                                    precision    recall  f1-score  support
War/Terror                           0.442804  0.941176  0.602258    255.0
Conspiracy Theory                    0.347368  0.733333  0.471429     45.0
Education                            0.307692  0.923077  0.461538     13.0
Election Campaign                    0.806452  0.757576  0.781250     33.0
Environment                          0.333333  0.714286  0.454545     14.0
Government/Public                    0.596288  0.883162  0.711911    291.0
Health                               0.297101  0.891304  0.445652     46.0
Immigration/Integration              0.296296  0.888889  0.444444     36.0
Justice/Crime                        0.357341  0.941606  0.518072    137.0
Labor/Employment                     0.217822  0.785714  0.341085     28.0
Macroeconomics/Economic Regulation   0.194079  0.951613  0.322404     62.0
Media/Journalism                     0.100000  0.958333  0.181102     48.0
Religion                 

/media/bruno/0d2f61d2-2b9c-4043-9a46-8e4dfe74fc95/bruno/Documents/ip6-twitter-disinformation/.conda/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:895: UserWarning: unknown class(es) ['', '(War/Terror', '0.5', '100% confidence', '24 February 2016', '48% grade', '5 votes', '50%', '50% confidence', '70% confidence', '9:47 pm', 'Accident', 'Arts/Culture', 'August 26', 'Brexit,', 'Club', 'Club Amnesia', 'Club)', 'Competition', 'David Squires wrote about the recent events in football.', 'Death', 'Debate/Interview', 'December 14', 'December 23rd 2015', 'Diplomacy', 'Entertainment', 'Event', 'Google', 'Information War', 'InternationalCoalition', 'Investment', 'Iran', 'Kurdistan', 'Manufacturing', 'Microsoft', 'Military', 'Military Intervention', 'Murder', 'Music', 'None of these are appropriate responses as they dont represent any of the intended tasks.', 'None of these platforms specifically mentioned.', 'None of these topics seem to apply to the input.', 'None of these topics 

## Metrics for multilabel but only the first class predicted is used

In [24]:
df2 = pd.read_csv("../data/vicuna_4bit/lora/multilabel_without_context_v01/test_generic_test_0.csv")
vicuna_lora_multilabel_without_context_v01_df = df2#pd.concat([df1, df2])
# Cleaning and extracting multilabel classes
vicuna_lora_multilabel_without_context_v01_df['annotations'] = vicuna_lora_multilabel_without_context_v01_df['annotations'].apply(lambda x: extract_multilabel_list(x, classes))
vicuna_lora_multilabel_without_context_v01_df['testmultilabel_without_context_v01'] = vicuna_lora_multilabel_without_context_v01_df['testmultilabel_without_context_v01'].apply(lambda x: extract_multilabel_list_only_first_class(x, classes))

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report

# Transform the labels into binary format
mlb = MultiLabelBinarizer(classes=classes)

# Prepare ground truth and predictions
y_true = mlb.fit_transform(vicuna_lora_multilabel_without_context_v01_df['annotations'])
y_pred = mlb.transform(vicuna_lora_multilabel_without_context_v01_df['testmultilabel_without_context_v01'])

#import pandas as pd
from sklearn.metrics import classification_report

# Generate the classification report
report = classification_report(y_true, y_pred, output_dict=True, target_names=classes)

# Convert the report to a pandas DataFrame
df_report = pd.DataFrame(report).transpose()

print(df_report)

                                    precision    recall  f1-score  support
War/Terror                           0.953947  0.568627  0.712531    255.0
Conspiracy Theory                    0.478873  0.755556  0.586207     45.0
Education                            0.421053  0.615385  0.500000     13.0
Election Campaign                    0.750000  0.545455  0.631579     33.0
Environment                          0.700000  0.500000  0.583333     14.0
Government/Public                    0.781377  0.663230  0.717472    291.0
Health                               0.740741  0.434783  0.547945     46.0
Immigration/Integration              0.750000  0.250000  0.375000     36.0
Justice/Crime                        0.929293  0.671533  0.779661    137.0
Labor/Employment                     0.687500  0.392857  0.500000     28.0
Macroeconomics/Economic Regulation   0.833333  0.322581  0.465116     62.0
Media/Journalism                     0.750000  0.250000  0.375000     48.0
Religion                 

/media/bruno/0d2f61d2-2b9c-4043-9a46-8e4dfe74fc95/bruno/Documents/ip6-twitter-disinformation/.conda/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:895: UserWarning: unknown class(es) ['Israel/Palestine', 'Propaganda'] will be ignored
  warnings.warn(
/media/bruno/0d2f61d2-2b9c-4043-9a46-8e4dfe74fc95/bruno/Documents/ip6-twitter-disinformation/.conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [25]:
df2 = pd.read_csv("../data/vicuna_4bit/lora/multilabel_with_rules_v02/test_generic_test_0.csv")
vicuna_lora_multilabel_without_context_v01_df = df2#pd.concat([df1, df2])
# Cleaning and extracting multilabel classes
vicuna_lora_multilabel_without_context_v01_df['annotations'] = vicuna_lora_multilabel_without_context_v01_df['annotations'].apply(lambda x: extract_multilabel_list(x, classes))
vicuna_lora_multilabel_without_context_v01_df['testmultilabel_with_rules_v02'] = vicuna_lora_multilabel_without_context_v01_df['testmultilabel_with_rules_v02'].apply(lambda x: extract_multilabel_list_only_first_class(x, classes))

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report

# Transform the labels into binary format
mlb = MultiLabelBinarizer(classes=classes)

# Prepare ground truth and predictions
y_true = mlb.fit_transform(vicuna_lora_multilabel_without_context_v01_df['annotations'])
y_pred = mlb.transform(vicuna_lora_multilabel_without_context_v01_df['testmultilabel_with_rules_v02'])

#import pandas as pd
from sklearn.metrics import classification_report

# Generate the classification report
report = classification_report(y_true, y_pred, output_dict=True, target_names=classes)

# Convert the report to a pandas DataFrame
df_report = pd.DataFrame(report).transpose()

print(df_report)

                                    precision    recall  f1-score  support
War/Terror                           0.967949  0.592157  0.734793    255.0
Conspiracy Theory                    0.442857  0.688889  0.539130     45.0
Education                            0.363636  0.615385  0.457143     13.0
Election Campaign                    0.761905  0.484848  0.592593     33.0
Environment                          0.642857  0.642857  0.642857     14.0
Government/Public                    0.787755  0.663230  0.720149    291.0
Health                               0.818182  0.391304  0.529412     46.0
Immigration/Integration              0.750000  0.166667  0.272727     36.0
Justice/Crime                        0.911765  0.678832  0.778243    137.0
Labor/Employment                     0.647059  0.392857  0.488889     28.0
Macroeconomics/Economic Regulation   0.826087  0.306452  0.447059     62.0
Media/Journalism                     0.857143  0.250000  0.387097     48.0
Religion                 

In [7]:
df2 = pd.read_csv("../data/vicuna_4bit/lora/multilabel_no_context_v01_256_rank/test_generic_test_0.csv")
vicuna_lora_multilabel_without_context_v01_df = df2#pd.concat([df1, df2])
# Cleaning and extracting multilabel classes
vicuna_lora_multilabel_without_context_v01_df['annotations'] = vicuna_lora_multilabel_without_context_v01_df['annotations'].apply(lambda x: extract_multilabel_list(x, classes))
vicuna_lora_multilabel_without_context_v01_df['testmultilabel_no_context_v01_256_rank'] = vicuna_lora_multilabel_without_context_v01_df['testmultilabel_no_context_v01_256_rank'].apply(lambda x: extract_multilabel_list_only_first_class(x, classes))

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report

# Transform the labels into binary format
mlb = MultiLabelBinarizer(classes=classes)

# Prepare ground truth and predictions
y_true = mlb.fit_transform(vicuna_lora_multilabel_without_context_v01_df['annotations'])
y_pred = mlb.transform(vicuna_lora_multilabel_without_context_v01_df['testmultilabel_no_context_v01_256_rank'])

#import pandas as pd
from sklearn.metrics import classification_report

# Generate the classification report
report = classification_report(y_true, y_pred, output_dict=True, target_names=classes)

# Convert the report to a pandas DataFrame
df_report = pd.DataFrame(report).transpose()

print(df_report)

                                    precision    recall  f1-score  support
War/Terror                           0.939394  0.607843  0.738095    255.0
Conspiracy Theory                    0.561404  0.711111  0.627451     45.0
Education                            0.444444  0.615385  0.516129     13.0
Election Campaign                    0.826087  0.575758  0.678571     33.0
Environment                          0.727273  0.571429  0.640000     14.0
Government/Public                    0.760163  0.642612  0.696462    291.0
Health                               0.689655  0.434783  0.533333     46.0
Immigration/Integration              0.583333  0.194444  0.291667     36.0
Justice/Crime                        0.908257  0.722628  0.804878    137.0
Labor/Employment                     0.727273  0.285714  0.410256     28.0
Macroeconomics/Economic Regulation   0.880000  0.354839  0.505747     62.0
Media/Journalism                     0.769231  0.208333  0.327869     48.0
Religion                 

/media/bruno/0d2f61d2-2b9c-4043-9a46-8e4dfe74fc95/bruno/Documents/ip6-twitter-disinformation/.conda/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:895: UserWarning: unknown class(es) ['Journalism', 'Nature', 'Statistics'] will be ignored
  warnings.warn(
/media/bruno/0d2f61d2-2b9c-4043-9a46-8e4dfe74fc95/bruno/Documents/ip6-twitter-disinformation/.conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
df2 = pd.read_csv("../data/vicuna_4bit/lora/multilabel_with_rules_v02_256_rank/test_generic_test_0.csv")
vicuna_lora_multilabel_without_context_v01_df = df2#pd.concat([df1, df2])
# Cleaning and extracting multilabel classes
vicuna_lora_multilabel_without_context_v01_df['annotations'] = vicuna_lora_multilabel_without_context_v01_df['annotations'].apply(lambda x: extract_multilabel_list(x, classes))
vicuna_lora_multilabel_without_context_v01_df['testmultilabel_with_rules_v02_256_rank'] = vicuna_lora_multilabel_without_context_v01_df['testmultilabel_with_rules_v02_256_rank'].apply(lambda x: extract_multilabel_list_only_first_class(x, classes))

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report

# Transform the labels into binary format
mlb = MultiLabelBinarizer(classes=classes)

# Prepare ground truth and predictions
y_true = mlb.fit_transform(vicuna_lora_multilabel_without_context_v01_df['annotations'])
y_pred = mlb.transform(vicuna_lora_multilabel_without_context_v01_df['testmultilabel_with_rules_v02_256_rank'])

#import pandas as pd
from sklearn.metrics import classification_report

# Generate the classification report
report = classification_report(y_true, y_pred, output_dict=True, target_names=classes)

# Convert the report to a pandas DataFrame
df_report = pd.DataFrame(report).transpose()

print(df_report)

                                    precision    recall  f1-score  support
War/Terror                           0.969325  0.619608  0.755981    255.0
Conspiracy Theory                    0.542373  0.711111  0.615385     45.0
Education                            0.409091  0.692308  0.514286     13.0
Election Campaign                    0.772727  0.515152  0.618182     33.0
Environment                          0.583333  0.500000  0.538462     14.0
Government/Public                    0.765873  0.663230  0.710866    291.0
Health                               0.800000  0.434783  0.563380     46.0
Immigration/Integration              0.625000  0.138889  0.227273     36.0
Justice/Crime                        0.920000  0.671533  0.776371    137.0
Labor/Employment                     0.750000  0.428571  0.545455     28.0
Macroeconomics/Economic Regulation   0.826087  0.306452  0.447059     62.0
Media/Journalism                     0.937500  0.312500  0.468750     48.0
Religion                 

/media/bruno/0d2f61d2-2b9c-4043-9a46-8e4dfe74fc95/bruno/Documents/ip6-twitter-disinformation/.conda/lib/python3.11/site-packages/sklearn/preprocessing/_label.py:895: UserWarning: unknown class(es) ['Radiation', 'War/Terror,'] will be ignored
  warnings.warn(
/media/bruno/0d2f61d2-2b9c-4043-9a46-8e4dfe74fc95/bruno/Documents/ip6-twitter-disinformation/.conda/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [10]:
# Select only 'f1-score' and 'support' for each label
df_report_selected = df_report.loc[classes, ['f1-score', 'support']]

# Convert the DataFrame to a LaTeX table
latex_table = df_report_selected.to_latex()

print(latex_table)

\begin{tabular}{lrr}
\toprule
 & f1-score & support \\
\midrule
War/Terror & 0.738095 & 255.000000 \\
Conspiracy Theory & 0.627451 & 45.000000 \\
Education & 0.516129 & 13.000000 \\
Election Campaign & 0.678571 & 33.000000 \\
Environment & 0.640000 & 14.000000 \\
Government/Public & 0.696462 & 291.000000 \\
Health & 0.533333 & 46.000000 \\
Immigration/Integration & 0.291667 & 36.000000 \\
Justice/Crime & 0.804878 & 137.000000 \\
Labor/Employment & 0.410256 & 28.000000 \\
Macroeconomics/Economic Regulation & 0.505747 & 62.000000 \\
Media/Journalism & 0.327869 & 48.000000 \\
Religion & 0.500000 & 11.000000 \\
Science/Technology & 0.470588 & 11.000000 \\
Others & 0.821561 & 271.000000 \\
\bottomrule
\end{tabular}



In [2]:
from tabulate import tabulate

models = [
    {
        "name": "No context - Classification only",
        "data": without_context_classification_only,
    },
    {
        "name": "No context - Elaboration first",
        "data": without_context_elaboration_first,
    },
    {
        "name": "With rules - Classification only",
        "data": with_rules_classification_only,
    },
    {
        "name": "With rules - Elaboration first",
        "data": with_rules_elaboration_first,
    }

]

for model in models:
    llm_utils.print_confusion_matrix(model["data"])

Confusion matrix for War/Terror:
[[56  9]
 [17 48]]

Confusion matrix for Conspiracy Theory:
[[54 11]
 [26 39]]

Confusion matrix for Education:
[[43 22]
 [ 9 56]]

Confusion matrix for Election Campaign:
[[42 23]
 [17 48]]

Confusion matrix for Environment:
[[36 29]
 [12 53]]

Confusion matrix for Government/Public:
[[32 33]
 [23 42]]

Confusion matrix for Health:
[[45 20]
 [16 49]]

Confusion matrix for Immigration/Integration:
[[52 13]
 [16 49]]

Confusion matrix for Justice/Crime:
[[28 37]
 [ 8 57]]

Confusion matrix for Labor/Employment:
[[47 18]
 [18 47]]

Confusion matrix for Macroeconomics/Economic Regulation:
[[63  2]
 [48 17]]

Confusion matrix for Media/Journalism:
[[31 94]
 [ 1  4]]

Confusion matrix for Religion:
[[97 31]
 [ 0  2]]

Confusion matrix for Science/Technology:
[]

Confusion matrix for War/Terror:
[[38 25]
 [ 6 59]]

Confusion matrix for Conspiracy Theory:
[[45 20]
 [26 38]]

Confusion matrix for Education:
[[24 41]
 [ 2 63]]

Confusion matrix for Election Camp

In [3]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 32.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.7/238.7 kB 30.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 66.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 63.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 39.3 MB/s eta 0:00:00
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19581 sha256=55c8fa74348198d4344fe896c217ac8ef74d5fe65a57d200f2c5438bc8c94f12
  Stored in directory: /home/bruno/.var/app/com.visualstudio.code/cache/pip/wheels/82/35/dc/f88ec71edf2a5596bd72a8fa1b697277e0fcd3cde83048

In [9]:
!pip install streamlit-components

ERROR: Could not find a version that satisfies the requirement streamlit-components (from versions: none)
ERROR: No matching distribution found for streamlit-components


In [6]:
import streamlit as st
import pandas as pd

# Title
st.title("Classification Report and Confusion Matrix Viewer")

# Model selection
model_names = [model["name"] for model in models]
selected_model = st.sidebar.selectbox("Select model", model_names)

# Class selection
class_options = classes
selected_class = st.sidebar.selectbox("Select class", class_options)

# Get the selected data
selected_model_data = None
for model in models:
    if model["name"] == selected_model:
        selected_model_data = model["data"]
        break

selected_confusion_matrix = selected_model_data["confusion_matrices"][selected_class]
selected_classification_report = selected_model_data["classification_reports"][selected_class]

# Display the confusion matrix and classification report
st.header("Confusion Matrix")
st.write(selected_confusion_matrix)

st.header("Classification Report")
st.write(pd.DataFrame(selected_classification_report).transpose())

In [7]:
!streamlit run report_viewer.py

Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Invalid value: File does not exist: report_viewer.py


In [5]:
without_contex_only_classification_df[without_contex_only_classification_df["Macroeconomics/Economic Regulation_pred"].notna()].copy().annotations.to_list()[0:10]

["['War/Terror']",
 "['Others']",
 "['Macroeconomics/Economic Regulation']",
 "['Government/Public', 'Macroeconomics/Economic Regulation']",
 "['Government/Public', 'Macroeconomics/Economic Regulation', 'Media/Journalism', 'War/Terror']",
 "['Government/Public', 'Macroeconomics/Economic Regulation']",
 "['Others']",
 "['Government/Public', 'Macroeconomics/Economic Regulation']",
 "['Education', 'Media/Journalism']",
 "['Justice/Crime']"]

In [4]:
without_contex_only_classification_df[without_contex_only_classification_df["Macroeconomics/Economic Regulation_pred"].notna()].copy().text.to_list()[0:10]

['RT @Ben_Jonson_1985: #Feylak Ar-Rahman militants located in #EasternGhouta do not release #civilians from the fighting zone. The use of a l…',
 'Women’s March Organizer Wants to “Take Vagina Away” from Female Genital Mutilation Survivor https://t.co/MmBm9WD6Ux https://t.co/T9rBUrOySf',
 'IMF has demanded to take credit apartments of Ukrainians\nhttp://t.co/2hcqYxfQHc #Ukraine',
 'Libertarianism is nothing but liberty for exploiters, racists, oppressors and parasites. Socialism is liberty for the people; the working class. https://t.co/uyuugJzTpq',
 '🇮🇱🇵🇸Israel is now experiencing a moment of heightened political, economic, and military power is no excuse to avoid dealing with the Palestinian issue, writes #valdaiclub expert Amos Yadlin.\n\nhttps://t.co/7IPavmsHJh',
 'Following the power cut crisis faced by Zimbabwe where electricity is only available between 10pm and 6am, people have adjusted their daily routines to make the most out of the electricity when it is available.\n\nRead f